In [43]:
df1 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-01.parquet")
df2 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-02.parquet")
df3 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-03.parquet")
df4 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-04.parquet")
df5 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-05.parquet")
df6 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-06.parquet")
df7 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-07.parquet")
df8 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-08.parquet")
df9 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-09.parquet")
df10 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-10.parquet")
df11 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-11.parquet")
df12 = spark.read.parquet("Files/nyctaxi_yellow_py/yellow_tripdata_2023-12.parquet")

StatementMeta(, 71aee94d-985c-4c76-8c60-b4ab2887a43b, 45, Finished, Available, Finished)

In [44]:
from pyspark.sql import DataFrame

# Assume df1, df2, ..., df12 are the DataFrames
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12]

# Combine all DataFrames using a loop
df_combined = dfs[0]
for df in dfs[1:]:
    df_combined = df_combined.union(df)




StatementMeta(, 71aee94d-985c-4c76-8c60-b4ab2887a43b, 46, Finished, Available, Finished)

In [45]:
df_lookup = spark.read.format("csv").option("header","true").load("Files/nyctaxi_lookup_zones/taxi_zone_lookup.csv")


StatementMeta(, 71aee94d-985c-4c76-8c60-b4ab2887a43b, 47, Finished, Available, Finished)

In [46]:
from pyspark.sql.functions import when, col, to_date, hour
from pyspark.sql.types import LongType, IntegerType, FloatType, StringType

# Map VendorID to vendor names (String -> VARCHAR)
df_combined = df_combined.withColumn(
    "vendor",
    when(col("VendorID") == 1, "Creative Mobile Technologies")
    .when(col("VendorID") == 2, "VeriFone")
    .otherwise("Unknown").cast(StringType())  # Cast to StringType
)

# Extract pickup and dropoff dates and hours
df_combined = df_combined.withColumn("tpep_pickup_date", to_date(col("tpep_pickup_datetime"), "yyyy-MM-dd")) \
                         .withColumn("tpep_dropoff_date", to_date(col("tpep_dropoff_datetime"), "yyyy-MM-dd")) \
                         .withColumn("tpep_pickup_time_hour", hour(col("tpep_pickup_datetime")).cast(IntegerType())) \
                         .withColumn("tpep_dropoff_time_hour", hour(col("tpep_dropoff_datetime")).cast(IntegerType()))

# Map payment_type to payment method (String -> VARCHAR)
df_combined = df_combined.withColumn(
    "payment_method",
    when(col("payment_type") == 1, "Credit Card")
    .when(col("payment_type") == 2, "Cash")
    .when(col("payment_type") == 3, "No Charge")
    .when(col("payment_type") == 4, "Dispute")
    .when(col("payment_type") == 5, "Unknown")
    .when(col("payment_type") == 6, "Voided Trip")
    .otherwise("Unknown").cast(StringType())  # Cast to StringType
)

# Perform the left join with df_lookup for PULocationID
df_combined = df_combined.alias("nty").join(
    df_lookup.alias("lu1"),
    col("nty.PULocationID") == col("lu1.LocationID"),
    "left"
).select(
    col("nty.*"),
    col("lu1.Borough").alias("pu_borough").cast(StringType()),  # Cast to StringType
    col("lu1.Zone").alias("pu_zone").cast(StringType())         # Cast to StringType
)

# Perform the left join with df_lookup for DOLocationID
df_combined = df_combined.alias("nty").join(
    df_lookup.alias("lu2"),
    col("nty.DOLocationID") == col("lu2.LocationID"),
    "left"
).select(
    col("nty.*"),
    col("lu2.Borough").alias("do_borough").cast(StringType()),  # Cast to StringType
    col("lu2.Zone").alias("do_zone").cast(StringType())         # Cast to StringType
)

# Cast passenger_count to LongType (Long Integer -> BIGINT)
df_combined = df_combined.withColumn("passenger_count", col("passenger_count").cast(LongType()))

# Cast trip_distance and total_amount to FloatType
df_combined = df_combined.withColumn("trip_distance", col("trip_distance").cast(FloatType())) \
                         .withColumn("total_amount", col("total_amount").cast(FloatType()))

# Final DataFrame
final_df = df_combined.select(
    "vendor",
    "tpep_pickup_date",
    "tpep_dropoff_date",
    "tpep_pickup_time_hour",
    "tpep_dropoff_time_hour",
    "pu_borough",
    "pu_zone",
    "do_borough",
    "do_zone",
    "payment_method",
    "passenger_count",
    "trip_distance",
    "total_amount"
)


StatementMeta(, 71aee94d-985c-4c76-8c60-b4ab2887a43b, 48, Finished, Available, Finished)

In [47]:
display(final_df)

StatementMeta(, 71aee94d-985c-4c76-8c60-b4ab2887a43b, 49, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 73239188-065a-4d7f-a376-93044698b72b)

In [48]:
final_df.write.format("delta").saveAsTable("nyctaxi_2023")

StatementMeta(, 71aee94d-985c-4c76-8c60-b4ab2887a43b, 50, Finished, Available, Finished)

In [49]:
df = spark.sql("SELECT count(1) FROM ProjectLakehouse.nyctaxi_2023")
display(df)

StatementMeta(, 71aee94d-985c-4c76-8c60-b4ab2887a43b, 51, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e4983cde-9b74-4e68-9c5e-bf25c1a3db55)